# Influx Relay

Example on how to send data into influx:
```
curl -i -XPOST 'http://localhost:8086/write?db=mydb' \
--data-binary 'cpu_load_short,host=server01,region=us-west value=0.64 1434055562000000000'
```

In [ ]:
import datetime
import time

import requests

import gmg.info
import gmg.net
import gmg.util

In [ ]:
grills = gmg.net.find_grills("192.168.42.0/24")
grill = grills[0]
grill

In [ ]:
gmg.info.get(grill)

In [ ]:
gmg.info.get(grill)

In [ ]:
influx_addr = "http://192.168.42.10:8086"

In [ ]:
import typing

class DataPoint(typing.NamedTuple):
    name: str
    tags: dict
    value: float
    time_ns: int

In [ ]:
def points_to_data(lines):
    return "\n".join(
        "{name},{tags} value={value} {ts}".format(
            name=l.name,
            tags=(",".join(f"{k}={v}" for k, v in l.tags.items())),
            value=l.value,
            ts=l.time_ns,
        )
        for l in lines
    )

In [ ]:
def write_influx(addr, data, db="kelvin"):
    url = addr + "/write"
    params = {"db": db}
    headers = {'Content-Type': 'application/octet-stream'}
    response = requests.post(url, params=params, data=data, headers=headers)
    return response

In [ ]:
import struct

In [ ]:
every_x = 5
while True:
    now = time.time()
    every_x_sleep_time = ((now // every_x) + 1) * every_x - now
    time.sleep(max(0.5, every_x_sleep_time))
    
    points = []
    try:
        data = gmg.info.get(grill)
    except (TimeoutError, RuntimeError) as exc:
        print(datetime.datetime.now().isoformat(), exc)
        time.sleep(1)
        continue
    
    now_ns = round(time.time_ns(), -8)
    
    for sensor in ["grill_temp", "grill_target", "probe1_temp", "probe2_temp"]:
        if data[sensor] > 600:
            continue
        points.append(DataPoint(
            name="temp_f",
            tags={"source": "gmg1", "sensor": sensor},
            value=data[sensor],
            time_ns=now_ns,
        ))
    
    for state in ["fire_state", "fire_state_pct"]:
        points.append(DataPoint(
            name="grill_fire",
            tags={"source": "gmg1", "sensor": state},
            value=data[state],
            time_ns=now_ns,
        ))
        
    
    points.append(DataPoint(
        name="grill_warnings",
        tags={"source": "gmg1"},
        value=data["warn_code"],
        time_ns=now_ns,
    ))

    data = points_to_data(points)
    write_influx(influx_addr, data)